In [ ]:
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine

api_key = "MD8gOfRQx4yZ0lS8CfXv6OkVtBLCIdw8"

param_dic = {
    "host"      : "db",
    "database"  : "weather",
    "user"      : "postgres",
    "password"  : "postgres"
}

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
engine = create_engine(connect)

locations = np.array([
    (25.8600,-97.4200),
    (25.9000,-97.5200),
    (25.9000,-97.4800),
    (25.9000,-97.4400),
    (25.9000,-97.4000),
    (25.9200,-97.3800),
    (25.9400,-97.5400),
    (25.9400,-97.5200),
    (25.9400,-97.4800),
    (25.9400,-97.4400),
])

locations_df = pd.DataFrame.from_records(locations, columns=["lat", "lon"])

In [11]:
for ix, row in locations_df.iterrows():
    lat = str(row["lat"])
    lon = str(row["lon"])
    if_exists = 'replace' if ix == 0 else 'append'

    url = "https://api.tomorrow.io/v4/timelines?apikey=" + api_key
    
    payload = {
        "location": tuple(row), # "42.3478, -71.0466",
        "fields": ["temperature", "windSpeed", "windDirection"],
        "units": "metric",
        "timesteps": ["1h"],
        "startTime": "nowMinus1d",
        "endTime": "nowPlus5d"
    }
    headers = {
        "accept": "application/json",
        "Accept-Encoding": "gzip",
        "content-type": "application/json"
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response_json = response.json()
    timelines = response_json["data"]["timelines"]
    timeline = timelines[0]
    intervals = timeline["intervals"]
    df = pd.DataFrame(intervals)
    print(df)
    df = pd.json_normalize(intervals)
    print(df)
    df.to_sql(
        'timeline', 
        con=engine, 
        index=False, 
        if_exists=if_exists
    )
    print("to_sql() done (sqlalchemy)")
    
    # url = "https://api.tomorrow.io/v4/weather/realtime?location=42.3478%2C%20-71.0466&units=metric&apikey" + api_key
    url = "https://api.tomorrow.io/v4/weather/realtime?location=" + lat + "%2C%20" + lon + "&units=metric&apikey" + api_key
    
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers)
    # values = [response.json()["data"]]
    
    # df = pd.DataFrame(values)
    df = pd.json_normalize(response.json())
    
    
    df.to_sql(
        'realtime', 
        con=engine, 
        index=False, 
        if_exists=if_exists
    )
    print("to_sql() done (sqlalchemy)")
    
    # json_str = json.dumps(response_json["data"]["timelines"], indent=2)
    
    # print(json_str)
    # print(response.json())

                startTime                                             values
0    2024-03-21T18:00:00Z  {'temperature': 23.13, 'windDirection': 232.19...
1    2024-03-21T19:00:00Z  {'temperature': 23.19, 'windDirection': 256.13...
2    2024-03-21T20:00:00Z  {'temperature': 23.63, 'windDirection': 257.5,...
3    2024-03-21T21:00:00Z  {'temperature': 24.19, 'windDirection': 298, '...
4    2024-03-21T22:00:00Z  {'temperature': 24.19, 'windDirection': 320, '...
..                    ...                                                ...
140  2024-03-27T14:00:00Z  {'temperature': 21.22, 'windDirection': 81.78,...
141  2024-03-27T15:00:00Z  {'temperature': 23.09, 'windDirection': 81.78,...
142  2024-03-27T16:00:00Z  {'temperature': 24.56, 'windDirection': 81.78,...
143  2024-03-27T17:00:00Z  {'temperature': 26.03, 'windDirection': 174.04...
144  2024-03-27T18:00:00Z  {'temperature': 27.5, 'windDirection': 174.04,...

[145 rows x 2 columns]
                startTime  values.temperature  value

KeyError: 'data'